## MIMIC-III Querying Tutorial

[Tutorial is available online](https://mimic.physionet.org/tutorials/intro-to-mimic-iii/)

Dr. Maria P. Frushicheva @ MIT

In [13]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# used to calculate AUROC/accuracy
from sklearn import metrics

%matplotlib inline

In [14]:
# create a database connection
sqluser = 'mimic' 
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

 ### Counting the Number of Patients in the Database

In [15]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
select count(subject_id)
from mimiciii.patients;
"""
data = pd.read_sql_query(query,con)
data

,count
0,46520


### Database Metadata for the Admissions Table

In [16]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT *
FROM admissions;
"""
data = pd.read_sql_query(query,con)
data.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_ioevents_data,has_chartevents_data
0,90,87,190659,2191-02-25 20:30:00,2191-04-25 15:18:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Private,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1
1,91,88,123010,2111-08-29 03:03:00,2111-09-03 14:24:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,,,,BLACK/AFRICAN AMERICAN,2111-08-29 01:44:00,2111-08-29 02:28:00,S/P MOTOR VEHICLE ACCIDENT-STABBING,0,1,1
2,92,89,188646,2185-06-17 05:22:00,2185-06-21 11:15:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1
3,93,91,121205,2177-04-23 00:08:00,2177-05-10 15:16:00,2177-05-10 15:16:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,,JEWISH,MARRIED,WHITE,2177-04-22 21:02:00,2177-04-23 04:03:00,FEVER,1,1,1
4,94,92,142807,2122-12-13 19:30:00,2123-03-04 13:47:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,,UNOBTAINABLE,,WHITE,NaT,NaT,NEWBORN,0,1,1


### Database Metadata for the Patients Table

In [17]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT *
FROM patients;
"""
data = pd.read_sql_query(query,con)
data.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,612,646,M,2128-01-05,None,None,None,0
1,613,647,M,2106-03-24,None,None,None,0
2,614,648,M,2139-07-13,None,None,None,0
3,615,649,M,2177-06-23,None,None,None,0
4,616,650,M,2051-04-15,2111-12-28 00:00:00,None,2111-12-28 00:00:00,1


### Database Metadata for the CharteventsTable

In [18]:
## Too long to execute!!!
#cur.execute('SET search_path to ' + schema_name)
#query = \
#"""
#SELECT *
#FROM chartevents;
#"""
#data = pd.read_sql_query(query,con)
#data.head()

 ### The Number of Patients in the Patients Table

In [19]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT COUNT(*)
FROM patients;
"""
data = pd.read_sql_query(query,con)
data

,count
0,46520


### The Distinct Values of Gender

In [20]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT DISTINCT(gender)
FROM patients;
"""
data = pd.read_sql_query(query,con)
data

,gender
0,M
1,F


### The Number of Female Patients

In [21]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT COUNT(*)
FROM patients
WHERE gender = 'F';
"""
data = pd.read_sql_query(query,con)
data

,count
0,20399


### The Numbers of Male and Female Patients

In [22]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT gender, COUNT(*)
FROM patients
GROUP BY gender;
"""
data = pd.read_sql_query(query,con)
data

,gender,count
0,M,26121
1,F,20399


### Count the Number of Patients Who Died in the Hospital

In [23]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT expire_flag, COUNT(*)
FROM patients
GROUP BY expire_flag;
"""
data = pd.read_sql_query(query,con)
data

,expire_flag,count
0,0,30761
1,1,15759
